# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\aryam\appdata\local\temp\pip-req-build-3uwrbt29
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fcf0d2197e0ff8fcc9b9ffb4bbab083a2deb61a6
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\aryam\appdata\local\temp\pip-install-d0e26z9i\e

  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\aryam\AppData\Local\Temp\pip-req-build-3uwrbt29'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\aryam\AppData\Local\Temp\pip-install-d0e26z9i\elegantrl_54d88fe1a0034e5aabc0707f54b72bed'


In [2]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

C:\Users\aryam\AppData\Local\Temp\ipykernel_41608\1185721934.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [3]:
ril_df_yf = yf.download(tickers = "RELIANCE.NS", start='2020-01-01', end='2024-02-10')

c:\ProgramData\anaconda3\envs\py310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [4]:
ril_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1360.645142,7002234
2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1383.809448,8855158
2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1385.476807,10492349
2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1353.344482,12259588
2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1374.165283,8341811


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [5]:
ril_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2024-02-10',
                                ticker_list = ['RELIANCE.NS']).fetch_data()

c:\ProgramData\anaconda3\envs\py310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (1020, 8)


In [6]:
ril_df_finrl.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-01,1387.957031,1396.277466,1376.527954,1360.645142,7002234,RELIANCE.NS,2
1,2020-01-02,1382.471069,1408.941040,1382.471069,1383.809448,8855158,RELIANCE.NS,3
2,2020-01-03,1401.671997,1409.581055,1392.528687,1385.476807,10492349,RELIANCE.NS,4
3,2020-01-06,1389.785767,1397.008911,1369.670410,1353.344482,12259588,RELIANCE.NS,0
4,2020-01-07,1388.871338,1403.043579,1383.842529,1374.165283,8341811,RELIANCE.NS,1


## Data for the chosen tickers

In [7]:
config_tickers.SENSEX_TICKERS

AttributeError: module 'finrl.config_tickers' has no attribute 'SENSEX_TICKERS'

In [8]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2024-02-10'

In [9]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

C:\ProgramData\anaconda3\envs\py310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\ProgramData\anaconda3\envs\py310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\ProgramData\anaconda3\envs\py310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[******************

Shape of DataFrame:  (111491, 8)


In [10]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.743890,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,42.737888,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.144922,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941090,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,30.950026,7117200,CAT,4


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [11]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


C:\ProgramData\anaconda3\envs\py310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (3801, 8)
Successfully added vix
Successfully added turbulence index


In [12]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [13]:
processed_full.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.743890,746015200.0,4.0,0.0,2.965559,2.638022,100.0,66.666667,100.0,2.743890,2.743890,39.189999,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,42.737888,6547900.0,4.0,0.0,2.965559,2.638022,100.0,66.666667,100.0,42.737888,42.737888,39.189999,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.144922,10955700.0,4.0,0.0,2.965559,2.638022,100.0,66.666667,100.0,15.144922,15.144922,39.189999,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941090,7010200.0,4.0,0.0,2.965559,2.638022,100.0,66.666667,100.0,33.941090,33.941090,39.189999,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,30.950026,7117200.0,4.0,0.0,2.965559,2.638022,100.0,66.666667,100.0,30.950026,30.950026,39.189999,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [14]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
26332


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [15]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')